# Top 50 startups

Startup is the first phase of the company, it is founded or lead by entrepreneurs who develop some product or service according to need of society. Their are many things we need for a proper Startup like:
* Business plan / Idea
* Location
* Raw materials
* Market knowledge
* Right people

But one of the most importing things is funding for proper growth and expansion of the startup. We need to look investors , and the investors look for business plan and the future of the startup and the field or tech on which startup is focusing on. <br>
Here we have dataset from [Kaggle](https://www.kaggle.com/sudalairajkumar/indian-startup-funding) which has the info regarding different startups, their funding and investors. We explore it to know :
* Which are some biggest funded startups?
* What are the most popular industries for startups?
* Who are the biggest investors?
* Which Location is preffered for startups? 



In [ ]:
cd /kaggle/input

In [ ]:
# importing libs
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import re
import seaborn as sns
from wordcloud import WordCloud
import datetime as dt
import squarify
from IPython.display import Image

In [ ]:
# reading the dataset
data=pd.read_csv('indian-startup-funding/startup_funding.csv',encoding='utf-8')

In [ ]:
data.head(10)

### Data exploring 

In [ ]:
data.info()

It can be seen from above that some of attributes have less value than others, it shows that their are some missing values. So,lets visualize which attributes have null value.

In [ ]:
#visualizing the null values for each attribute
plt.figure(figsize=(10,5)) 
sns.heatmap(data.isnull(),cmap='magma',yticklabels=False,cbar=False)
plt.show()

Their are many missing values in the Revenue attribute and in `Remarks` attribute. Inorder to further process , lets see what are some non-null value in it.

In [ ]:
#getting the unique value in Remark attribute.
data['Remarks'].unique()

### Data cleaning
Their are some strange words present in the dataset like: `\xc2` ,`\xa0` etc.. If we  search for it then we found those are the [unicode](https://www.codetable.net/hex/a0) symbols. <br>
Ex: "\xa0" represents 'no-break space'

In [ ]:
# checking other attributes have those unicode symbol or not
# i am specifying some the row from 150-200 for showing
data['Industry Vertical'].unique()[150:200]

So first we need to remove these unicde for making some meaninful out of it. We can remove these using `regex` pattern from string, but before removing them we need to handle the null values.

In [ ]:
#checking how many null values are their in `Amount in USD(Funding)` attribute
data[data['Amount in USD'].isnull()]

Their are any null value in `Amount in USD` attribute , their are many null values in other attributes, so we need to fill them as well.

In [ ]:
#filling null value with "others" for attributes other than 'Amount in USD'.
data[1:-2].fillna('others',inplace=True)

In [ ]:
data['Amount in USD'].unique()

The `Amount in USD`(Funding) attribute has *undisclosed* and *unknown* value so we can remove them if they are less.

In [ ]:
# checking how many values are presen with value undisclosed
data[(data['Amount in USD']=='undisclosed')]

In [ ]:
## checking how many values are presen with value unknown
data[(data['Amount in USD']=='unknown')]

In [ ]:
data[(data['Amount in USD']=='Undisclosed')]

The number of non numerical value for `Amount in USD` attribute are 7 , so we can remove them.

In [ ]:
data[data['Amount in USD'].isnull()]

Their are many null values in Funding attribute , i explore some of the sites to collect them like [crunchbase](https://www.crunchbase.com/) , but we have need the details regarding the investors , that how much a particular investor has fund in a particular startup , but the site does not contains it ,instead it contains the total funding a particular startup has , and the specific amount is not available on the other sites as well. Their are many ways to fill those values :
* By filling them with 0
* By filling them with avg <br>

But those would not help much , because the number of missing values are 1/3 of the total dataset and we try the model to predict the funding amount it would not work properly because we would have a lot of variance in it.So , instead we need to drop it for better prediction of funding amount.


In [ ]:
# drop rows with non numerical funding amount 
data.drop(data.loc[(data['Amount in USD']=='undisclosed') | (data['Amount in USD']=='Undisclosed') | (data['Amount in USD'].isnull()) | (data['Amount in USD']=='unknown')].index,inplace=True)

In [ ]:
data['Remarks'].fillna('None',inplace=True)

In [ ]:
#replacing unneccesary word with others
data['Investors Name'].replace('undisclosed','others',inplace=True)

In [ ]:
data.shape

In [ ]:
#checking wheather we have processed all the null values
data.isnull().sum()

We still remain with null values so remove them by droping null value from dataframe

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.shape

Now we can process those unicode symbol, here is the a example that how we regex to remove those symbols

In [ ]:
t="sequoia capital, rohit bansal, kunal bahl, sandeep tandon, kunal shah, zishaan hayath,\\\ abhi"
t=re.sub(r"\\+","",t)
t

To know more about the regex one can follow over [here](https://regexr.com)

In [ ]:
# defining func for the removing of unicodes from each cell of dataframe
def unicode(x):
    x=re.sub(r"\\x..","",x)     ## removing unicode as it contains 2 char followed by '\x'
    x=re.sub(r"\\","",x)     ## removing the backslashes
    x=x.lower()                   ## lower the strings
    # print(x)
    return x.strip()

In [ ]:
# process each cell of dataframe to remove unicode
for col in data.columns[1:]:
    data[col]=data[col].apply(unicode)

In [ ]:
data

Their are many char in `Amount in USD` like ',' ,  '+'  and 'n/a' , inorder to convert them to float datatype we need to remove these character.

In [ ]:
# removing unnecessary char
data["Amount in USD"] = data["Amount in USD"].apply(lambda x:(str(x).replace(",","").replace("+","").replace(r'n/a',"others")))


In [ ]:
data=data[data['Amount in USD']!='others']
data['Amount in USD']=pd.to_numeric(data['Amount in USD']) #converting funding attribute to numeric(float) type 

In [ ]:
data['Date dd/mm/yyyy'].unique()

While converting the `Date` attribute from object to  datetime  i get some error regaring the char ,so i check the values we have in `Date` attribute. As we can see some of the dates do not have '/' some have '.' in it.<br>
Ex:  05/072018  ,   22/01//2015

In [ ]:
#removing the '.' and replacing them with '/'
data['Date dd/mm/yyyy']=data['Date dd/mm/yyyy'].apply(lambda x: x.replace(".","/"))

In [ ]:
#converting some dates to their proper format
data['Date dd/mm/yyyy'].replace('22/01//2015','22/01/2015',inplace=True)
data['Date dd/mm/yyyy'].replace('01/07/015','01/07/2015',inplace=True)
data['Date dd/mm/yyyy'].replace('05/072018','05/07/2018',inplace=True)

### Data preprocessing

In [ ]:
# getting some feature like 'date','month' and 'year' from Date.
data['year']=pd.to_datetime(data['Date dd/mm/yyyy'],format='%d/%m/%Y').dt.year
data['month']=pd.to_datetime(data['Date dd/mm/yyyy'],format='%d/%m/%Y').dt.month
data['day']=pd.to_datetime(data['Date dd/mm/yyyy'],format='%d/%m/%Y').dt.day

In [ ]:
data["yearmonth"] = (pd.to_datetime(data['Date dd/mm/yyyy'],format='%d/%m/%Y').dt.year*100)+(pd.to_datetime(data['Date dd/mm/yyyy'],format='%d/%m/%Y').dt.month)

In [ ]:
# checking what kind of value does the 'City Location' have
data['City  Location'].unique()

Their are many cities which has 2 places mentioned , we can took one of them for making for visualizing as almost all of them contains city on left side of backslash and country on right side , so we go with cities as other locations are also the name of some cities and this is the same case for `InvestmentnType`.<br>
Ex: Seed/ Angel Funding, here the angel and seed Funding are same type of funding , more details can be found [here](https://www.investopedia.com/terms/a/angelinvestor.asp).
So we can remove the term seed from it.


In [ ]:
# getting angel type of funding 
data['InvestmentnType']=data['InvestmentnType'].apply(lambda x: x.split("/")[-1])

In [ ]:
# getting the city name where we have 2 names available
data['City  Location']=data['City  Location'].apply(lambda x: x.split("/")[0].strip())

from above we can also see that some of the cities like gurgaon and gurugram points to same locations, so we can map it to same kind of name.

In [ ]:
## function to change those common city names
def map(x):
    if x=='gurugram':
        return 'gurgaon'
    elif x=='bengaluru':
        return 'bangalore'
    else:
        return x
data['City  Location']=data['City  Location'].apply(map)

In [ ]:
data

### Visualization
To answer our first question that **what are top 50 funded startup?** we visualise them by barplot

In [ ]:
## creating new list to having startups with their total funding
topfiftystart=[]
for startup in data['Startup Name'].unique():
    df=data[data['Startup Name']==startup]    ## get the dataframe for each startup
    sum=np.sum(df['Amount in USD'])         ## sum total funding of startup
    topfiftystart.append([startup,sum])

startup=pd.DataFrame(topfiftystart,columns=['startup','Revenue'])  #converting the list to dataframe and sort them by the fundin amount
startup.sort_values(by='Revenue',ascending=False,inplace=True)

In [ ]:
startup

In [ ]:
data.shape

In [ ]:
## diving amount by 1M to show data in term of Millions
startup['Revenue']/=1000000
startup=startup[:51]

In [ ]:
## visualising the amount spend on the top 50 startups
plt.figure(figsize=(20,50))
ax = sns.barplot(x="Revenue", y="startup",data=startup, palette="husl",linewidth = 2,label='big')
for i,j in enumerate(startup["Revenue"]):
    ax.text(10, i, j, weight="bold", color = 'black', fontsize =15,ha='left')
plt.title("50 most revenue  startups ",fontsize=20)
ax.set_xlabel(xlabel = 'Amount(in Millions)', fontsize = 20)
ax.set_ylabel(ylabel = 'Startup company', fontsize = 30)
plt.show()
del startup

We can see the top 50 highest funded startups and Flipkart is one of the biggest startup with the highest funded amount.

Now will see who are the biggest investors in the market.

In [ ]:
company=data.copy()

In [ ]:
company.sort_values(by='Amount in USD',ascending=False,inplace=True) ## sort the investor according to their fundings

In [ ]:
company['Amount in USD']/=1000000
company=company[company['Investors Name']!='others']
company=company[:10]

In [ ]:
# visualizing the biggest investors
plt.figure(figsize=(15,10))
ax = sns.barplot(x="Amount in USD", y="Investors Name",data=company, palette="husl",linewidth = 2,label='big')
for i,j in enumerate(company["Amount in USD"]):
    ax.text(10, i, j, weight="bold", color = 'black', fontsize =15,ha='left')
plt.title("10 bigest investor ",fontsize=20)
ax.set_xlabel(xlabel = 'Amount(in Millions)', fontsize = 20)
ax.set_ylabel(ylabel = 'Investors Name', fontsize = 30)
plt.show()
del company

We can see the biggest investor for the startups and the biggest investors are `Westbridge capital` and `softbank`.


In [ ]:
## visualizing the most valued industries prefered for funding 
industry = data['Industry Vertical'].value_counts().head(10)
plt.figure(figsize=(15,8))
sns.barplot(industry.index, industry.values, alpha=0.9, palette="husl")
plt.xticks(rotation=45)
plt.xlabel('Industry of startups', fontsize=12)
plt.ylabel('freq of industries for the startup', fontsize=12)
plt.title("Most valued industries for funding", fontsize=16)
plt.show()

Now we look wo are the most frequent investors in the market. If we see in the investors name then we can observe that their are some name like undisclosed, undisclosed investos , which represents nothing so we can put them under the category of 'others'.

In [ ]:
## investor func to name undisclosed investors to others
def investor(x):
    x=x.strip()
    if x=='undisclosed' or x=='undisclosed investors' or x=='undisclosed investor':
        return 'others'
    else:
        return x
data["Investors Name"]=data["Investors Name"].apply(investor)

In [ ]:
# getting the names of investors
names = data[data['Investors Name']!='others']["Investors Name"]

In [ ]:
# creating the wordcloud to observe the most frequent investor
wordcloud = WordCloud(max_font_size=50, width=600, height=300, background_color='cyan').generate(' '.join(names))
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.title("Frequent investors", fontsize=35)
plt.axis("off")
plt.show()

From above we can see the investors who has invested more than one startups are :
* venture partner
* sequoia capital
* accesl partner

In [ ]:
#in which sector there are most startups
d=data[data['Industry Vertical']!='others']['Industry Vertical'].value_counts().head(6)
explode = (0.1, 0, 0, 0,0,0)
fig1,ax1=plt.subplots(figsize=(20,10))

ax1.pie(d.values,explode=explode, labels=d.index,autopct='%1.1f%%', shadow=True, startangle=140)
ax1.axis('equal')
plt.title("Famous industries of startup",fontsize=30)
plt.show()

The most of the startups are based on:
* Consumer internet
* Technology
* e-commerce industries

Now we also look at what are the most preferred locations for the startups.

In [ ]:
# plot the locations according to the funding given to the starups
plt.figure(figsize=(17,12))
mean_amount = data.groupby('City  Location').sum()["Amount in USD"].sort_values(ascending=False).head(15)
squarify.plot(sizes=mean_amount.values,label=mean_amount.index, value=mean_amount.values)
plt.title('Distribution of Startups across Top cities')

Banglore is  the most preffered industry for the startups , that's why we can also say that Banglore is the IT hub,  the location also matters so one should also think about the location in terms of funding and kind of industry statups are working on.

**Now we have answer to all the question we have mentioned at the starting**.

In [ ]:
corr = data.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)

from above we can see that `Amount in USD` has less correlation with other attributes , so we need to find some more features for training.

### Feature engineering

In [ ]:
data['Investors Name'].unique()

If we look at the names of investors then we can find that their are more than one investor for some of the startups and they are separated by ',' , so we can create a list of those names as to extract unique names form it.

In [ ]:
data.columns

To predict the Funding amount ,we need to have the features with numerical value because the system only understands the numbers.

So we have the attribute `City  Location` having the name of location for each startups. So we need to convert it to numerical value. If we look at the investors name then each of the startup have more than one investor, but need to focus more on the revenue and location. So , we will need to aggregate all the revenue for each startup and each need to list all the location for it.
Ex:

|Startup|Location|revenue|
|---|---|---|
|byju's|banglore|2000|
|mamearth|gurgram|3500|
|pandos|chennai|4000|
|byju's|gurugram|3000|
|mamearth|gurgram|2500|
|...|...|...|


then after aggregating the total revenue and list all the location we would left with:


|Startup|Location|revenue|
|---|---|---|
|byju's|\[banglore,gurugram\]|5000|
|mamearth|gurgram|6000|
|pandos|chennai|4000|
|...|...|...|

Then we can convert the list of location to [one hot enconding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to convert them to numerical value.


In [ ]:
### aggregate the revenue as per the startup name 
## get the list of all locations 
for i,row in data.iterrows():
    data.at[i,'Location']=list(set(data[data['Startup Name']==row['Startup Name']]['City  Location'].values))
    data.at[i,'Revenue']=float(np.sum(data[data['Startup Name']==row['Startup Name']]['Amount in USD'].values))
    # print(row['Startup Name'])

In [ ]:
## remove the duplicate startups rows
data.drop_duplicates(subset='Startup Name',keep=False,ignore_index=True,inplace=True)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [ ]:
##convert location to the one-hot  encoding
res = pd.DataFrame(mlb.fit_transform(data['Location']),columns=mlb.classes_,index=data['Location'].index)

In [ ]:
res

In [ ]:
data=pd.concat([data,res],axis=1)  ## concatinating both the dataframe

In [ ]:
data

In [ ]:
# ## splitting the names of invstors separated by ','
# data['investors']=data['Investors Name'].apply(lambda x: set(x.split(',')))

In [ ]:
# ## creating vocab for the unique names of investors
# investor=[]
# for each in data['investors']:
#     for i in each:
#         if i!="":
#             investor.append(i.strip())

# investor=list(set(investor))
# print(investor[:10])  

In [ ]:
# # creating ranks for the names of investors vocab 
# investors={}
# i=1
# for inv in investor:a
#     investors[inv]=i
#     i+=1


In [ ]:
# ## creating func val for mapping the sum of rank of investors name
# def val(x):
#     v=0
#     for i in x:
#         if i:
#             v+=investors[i.strip()]
#     return v

In [ ]:
# ## applying func to map the names as the sum of rank of investors name
# data['investor val']=data['investors'].apply(val)

We need to now map the `Subvertical` to numerical value.
As it contains small discription of the Industry ,so we need to extract some important terms which has more value in a sentence.<br>
Ex: Online Marketplace For Mother and Babycare Products<br>
Here can see the word babycare , Marketplace and mother has more weightage than other words. One of the example is `google search engine`.<br>
To find this weightage of term we are using TF*IDF algo.<br>
1. TF(term frequencey)(t,d)=count of word t in the document(d)/number of words in d <br>
2. IDF(inverse document frequencey)= $\log (\frac{N}{df+1})$<br>
where N=occurence of term t in document set N <br>
df=occurences of term t in document set N.

TF-IDF=TF(t,d) x $\log (\frac{N}{df+1})$<br>
For more details click [here](https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089)

I am using Sklearn for creating TF-IDF vectors.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
data.columns

In [ ]:
v = TfidfVectorizer(stop_words='english',analyzer='word',max_features=500) ## vector should create vector for 800 most weightage words
x = v.fit_transform(data['SubVertical'])

In [ ]:
data.reset_index(drop=True,inplace=True) 

In [ ]:
t=x.toarray()   # converting vector list to array

In [ ]:
dt=pd.DataFrame(t)

In [ ]:
dt

In [ ]:
df=pd.concat([data,dt],axis=1)   ## concatenating the tf-idf vectors with prevoius dataframe

We got remain with attributes : `Startup Name` ,`Industry Vertical`, `City  Location` and `InvestmentnType`. We can map the values to number like we have done with ranks dict created above for the investors name. <br>
I am using `LabelEncoder` for this.

In [ ]:
from sklearn import preprocessing

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
df['Startup Name']=le.fit_transform(df['Startup Name'])
df['Industry Vertical']=le.fit_transform(df['Industry Vertical'])
df['InvestmentnType']=le.fit_transform(df['InvestmentnType'])


In [ ]:
## drop the non-featured attributes
df.drop(labels=['Date dd/mm/yyyy','Industry Vertical','SubVertical','Investors Name','yearmonth','Remarks','Amount in USD','Location','City  Location'],inplace=True,axis=1)

In [ ]:
df

In [ ]:
df['Revenue']/=1000000


## Training model

As we need to predict the `Amount in USD` which is continuous value ,we can predict them with regression. Consider this Image:

In [ ]:
Image("/kaggle/input/images/linear.png",width = 600, height = 300)




The line fitting those datapoints has the equation , J(y_pred)= $\theta * X +b$<br>
J is also known as loss function<br>
b = intercept<br>
X= feature from the datapoints<br>
$\theta$= slope of the line

We can see that the line does not fit all the datapoints , and we can their is some error between each points and the line . We can find the total error by sum of square of those error i.e. $S=(y\_given^{i}-J^{i})^2$ <br>
and we use the square term because some of points above the line might have positive error and some of the points below the line has negaive error and we sum them we might get the actual error or the total error get decreased.<br>
Now the goal is that we use S to update the theta such that Sum of square error could be minized: <br>
$\theta= \theta - LR* \frac{dS}{d\theta}$<br>
$b=b-LR* \frac{dS}{db}$<br>
LR is learning rate at which the update the weights , one can visualise them as the  we do the [gradient descent](https://blog.clairvoyantsoft.com/the-ascent-of-gradient-descent-23356390836f) to find the global optimal. 

Some of these line get overfit the datasets i.e. high variance, one can also understand it like the Training model get well trained over the given dataset but might not perform well for new coming datapoints. So we address this problem using **Rigde regression**.

### Rigde regression (L2 regularization)
Consider the Image:



In [ ]:
Image("/kaggle/input/images/ridge.png",width = 600, height = 300)

Consider those red datapoints as the poitns from the training set , and the red line fit perfectly to those points but if try to predict new points(green points) then we get high variance from the red line. So we try to reduce this variance by introducing some bias to the sum of square error and get the new line which is represented by blue colour by reducing some variance.This provides a better long term prediction params. <br>
This introducing of some bias is also known as penalty(regularization term), <br>represented as =
$\lambda * {\theta}^2$<br>
$\theta$=penalty to loss function<br>
$\lambda$ represents how severe the penalty is

So the now our new loss function would become:<br>
$J= (y\_given-(\theta * X +b))^2+\lambda * {\theta}^2 $


In [ ]:
Image("/kaggle/input/images/alpha.png",width = 600, height = 300)

lambda is represented as alpha in the image.

As increase the lambda the slope of the regression line reduces and becomes more horizontal and the model becomes less sensitive to variation of the independent variable X , thus slope of the line is reduced and a small change in the X would have less impact in the change of Y and we can avoid the overfitting of the model.

Now the question arises how do chose the value of lambda?<br>
Ans:For this we will use k-fold cross validation.

### K-fold cross validation

Condider the dataset:

|sr.|X|Y|
|---|---|---|
|1|101|2|
|2|120|3|
|3|135|3|
|...|...|...|
|...|...|...|
|1000|256|8|

Lets say we divid the dataset into 5 parts i.e. k=5, so get the partion as:<br>
1st set have datapoints from \[1,2...200\]<br>
2nd set have datapoints from \[201,202...400\]<br>
3rd set have datapoints from \[401,402...600\]<br> 
4th set have datapoints from \[601,602...800\]<br>
5th set have datapoints from \[801,802...1000\]

Now make set 1-4 as train set and 5 as the test set and do this 4 more time for other sets as well, finally we would train our model over all the combination of test and train set and we get error from each of the combination say: E1,E2,E3,E4 and E5. We aggregate those results can conclude the final value of lambda which has the less error value in k-fold cross validation.

I am using sklearn for ridge regression

In [ ]:
## importing the libraries
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import mean
from numpy import std
from numpy import absolute
import pickle

In [ ]:
model= Ridge(normalize=True,alpha=0.1) ## normalize the value for faster convergence while gradient descent
cv = RepeatedKFold(n_splits=20, n_repeats=3, random_state=32)   # k-fold cross validation with k=20
scores = cross_val_score(model,df.loc[:, df.columns != 'Revenue'],df['Revenue'], scoring='neg_median_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
# pickle.dump(model,open('/kaggle/input/indian-startup-funding/ridge_reg.pkl','wb'))
with open("/kaggle/ridge_reg.sav",mode='wb') as model_f:
    pickle.dump(model,model_f)

We get the overall mean absolute error of 16.897 from our model.


Their are many features available after converting the text data to numerical value, their might be some features which are less useful while training the model ,so we can neglect of those features by introducing Lasso Regression.

### Lasso Regression(L1 Regression)
It is same as the Ridge Regression except the difference is that it uses norm instead of square for penalty term i.e:<br>
$\lambda * {|\theta|}$<br>
So our final function would be :
$J= (y\_given-(\theta * X +b))^2+\lambda * {|\theta|} $

How it affect the features?<br>
Consider The training model : <br>


In [ ]:
Image("/kaggle/input/images/data.png",width = 1000, height = 300)


When we train the model the using Ridge Regression ,it has the term ${\theta}^2$ as the penalty , and if see from above image we have many features in it. Some of the feature like `Location`,`startup name` might have value for our training model but some features like `489`,`402` ,`Sr.no.` etc. might not have much value/effect for our training model , so as we increase the lambda value the penalty for the Ridge Regression goes Asymptotically to 0 ,and those features remain in out training set with some less impact on the sum of squares error but the **Lasso Regression** has the norm term as penalty and as we increase the lambda then those less important features goes straight forward to 0 and those features has no effect for the trainig model and we obtain the model with less sum of squares of error than the Ridge Regression . Thus Lasso Regression perform better than Ridge Regression by remove the effect of some non useful features for the training model.

In [ ]:
#importing lib for lasso regression
from sklearn import linear_model

In [ ]:
clf = linear_model.Lasso(alpha=0.1,normalize=True)
cv = RepeatedKFold(n_splits=20, n_repeats=3, random_state=32)
scores = cross_val_score(clf,df.loc[:, df.columns != 'Revenue'],df['Revenue'], scoring='neg_median_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

From above we can see that the overall Mean absolute error by Lasso Regression is less than the Ridge Regression.

In [ ]:
# pickle.dump(clf,open('lasso_reg.pkl','wb'))
with open("/kaggle/lasso_reg.sav",mode='wb') as model_f:
    pickle.dump(clf,model_f)

Conclusion:
* The most funded revenue startups are `Flipkart` and `Rapido ride taxi`
* The highest funded investores are `Westbridge capital` and `softbank`.
* The industry most preferred by investors for funding the startup are `Consumer internet` and `Technology`. The new startups should focus on these field for better funding and growth.
* And the most preffered location for the startups is Banglore .